# Echantillonnage au niveau du district / Sampling at health district level

Pour évaluer des interventions de santé publique tel que vaccination de routine et même les campagnes, les données administratives sont utilisées. Les enquêtes de couvertures sont généralement faites à l'échelle nationale.   
Le présent article étudiera la pertinence et la faisabilité d'enquête de couverture au niveau de district.        
Les différents types d'échantillonnages probabilistes seront abordés:   

1.	Echantillonnage aléatoire simple
2.	Echantillonnage systématique
3.	Echantillonnage stratifié (proportionnel ou à nombre égal)
4.	Echantillonnage pondéré
5.	Echantillonnage en grappe   

Le dataset qui a servi de base de sondage est le résultat d'un dénombrement réalisé dans une aire de santé. Dénombrement fait dans le cadre des activités liées à l'éradication de la filariose lymphatique par le traitement de masse. Ce dataset contient 1918 lignes qui correspondent à autant de ménages. Les informations collectées sont le nombre d'individus par ménages par sexe et par tranche d'âge (moins d'un an, 1-5 ans, 5-15 ans et plus de 15 ans). Ces ménages sont repartis entre les 8 "Lawanats" (chefferie de 2e degré) et les 82 quartiers (chefferie de 3e degré) qui composent cette aire de santé.    

Les métriques que nous évaluerons sont:
- La taille moyenne des ménages
- La proportion des ménages sans enfant de moins d'un an
- La proportion des enfants de moins d'un par rapport à la population

Les outils de data analyse que sont python et ses librairies (pandas et numpy) ont été utilisés.

#### Importation des librairies

In [1]:
import pandas as pd
import numpy as np
import random

#### Importation des données (base de sondage)

In [2]:
population = pd.read_excel('Base_sondage_Vele_all.xlsx')

#### Exploration de données

In [3]:
population.head()

,LAWANAT,QUARTIER,N° MENAGE,< 1 MASC,1 - 4 MASC,5- 15 MASC,> 15 MASC,< 1 FEM,1 - 4 FEM,5 - 15 FEM,> 15 FEM
0,AGOLLA,AGOLLA DOUNA,1,0,1,1,3,0,2,3,4
1,AGOLLA,AGOLLA DOUNA,2,0,1,2,2,0,0,1,2
2,AGOLLA,AGOLLA DOUNA,3,0,1,3,1,0,1,0,3
3,AGOLLA,AGOLLA DOUNA,4,0,0,1,3,1,0,0,1
4,AGOLLA,AGOLLA DOUNA,5,0,0,2,2,0,1,3,2


In [4]:
population.shape

(1918, 11)

In [5]:
population.dtypes

LAWANAT       object
QUARTIER      object
N° MENAGE      int64
< 1 MASC       int64
1 - 4 MASC     int64
5- 15 MASC     int64
> 15 MASC      int64
< 1 FEM        int64
1 - 4 FEM      int64
5 - 15 FEM     int64
> 15 FEM       int64
dtype: object

In [6]:
population.describe()

,N° MENAGE,< 1 MASC,1 - 4 MASC,5- 15 MASC,> 15 MASC,< 1 FEM,1 - 4 FEM,5 - 15 FEM,> 15 FEM
count,1918.000000,1918.000000,1918.000000,1918.000000,1918.000000,1918.000000,1918.00000,1918.000000,1918.000000
mean,15.220021,0.168926,0.638165,1.570386,2.261210,0.168405,0.61366,1.539103,2.571429
std,10.876486,0.493710,0.849955,1.512942,1.538946,0.442055,0.80709,1.406922,1.645993
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,6.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,1.000000
50%,13.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.00000,1.000000,2.000000
75%,22.000000,0.000000,1.000000,2.000000,3.000000,0.000000,1.00000,2.000000,3.000000
max,53.000000,10.000000,6.000000,10.000000,10.000000,6.000000,6.00000,12.000000,13.000000


In [7]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1918 entries, 0 to 1917
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   LAWANAT     1918 non-null   object
 1   QUARTIER    1918 non-null   object
 2   N° MENAGE   1918 non-null   int64 
 3   < 1 MASC    1918 non-null   int64 
 4   1 - 4 MASC  1918 non-null   int64 
 5   5- 15 MASC  1918 non-null   int64 
 6   > 15 MASC   1918 non-null   int64 
 7   < 1 FEM     1918 non-null   int64 
 8   1 - 4 FEM   1918 non-null   int64 
 9   5 - 15 FEM  1918 non-null   int64 
 10  > 15 FEM    1918 non-null   int64 
dtypes: int64(9), object(2)
memory usage: 165.0+ KB


In [8]:
population.columns

Index(['LAWANAT', 'QUARTIER', 'N° MENAGE', '< 1 MASC', '1 - 4 MASC',
       '5- 15 MASC', '> 15 MASC', '< 1 FEM', '1 - 4 FEM', '5 - 15 FEM',
       '> 15 FEM'],
      dtype='object')

#### Liste des lawanats

In [9]:
population.LAWANAT.unique()

array(['AGOLLA', 'GUEMERE', 'GUIA', 'MERINGUE 1', 'MERINGUE 2',
       'MERINGUE 3', 'WIDI', 'WIDIGUE'], dtype=object)

#### Nombre de ménages par lawanat

In [10]:
population.LAWANAT.value_counts(dropna=False)

WIDI          443
WIDIGUE       357
AGOLLA        273
MERINGUE 2    204
MERINGUE 1    192
GUIA          186
GUEMERE       167
MERINGUE 3     96
Name: LAWANAT, dtype: int64

#### Liste des quartiers

In [11]:
population.QUARTIER.unique()

array(['AGOLLA DOUNA', 'BERE CENTRE', 'BIDWERE 1', 'BIDWERE 2',
       'BIDWERE 3', 'BODINGA', 'BOUROU 1', 'BOUROU 2', 'DANAY 1',
       'DANAY 2', 'KOMKOSSAYE 1', 'KOMKOSSAYE 2', 'MAGA',
       'BOURMI NGUEVELEKE', 'BOURMI RAVANTONGAI', 'DAGADAGA',
       'DANAY DOUVOUGOUNI', 'DANAY VOULHANA', 'GUEMERE CENTRE', 'HIRAY',
       'KOUSSOUK', 'YOUNA', 'GUIA BACKSARA', 'GUIA BIDWERE',
       'GUIA MOKOLOM', 'GUIA RAKAYE', 'GUIA WIDI', 'BACKSARA', 'DARAM',
       'DEKDEREK', 'DIGUIMBE', 'DJO', 'GOHO', 'LANGUI', 'MARANDJAM',
       'MOKO', 'NOULTONGAYE CENTRE', 'SAKATONGAI', 'FITIMLAYE 1',
       'FITIMLAYE 2', 'GANDJANG', 'NETONGAI', 'SOUALAYE', 'TCHONG 1',
       'TCHONG 2', 'BANGAL', 'GOLONGOBOYE 1', 'GOLONGOBOYE 2',
       'GOUNOURGOU', 'MASSOUANG 1', 'MOUNHA ', 'DINGUE', 'DJALWA',
       'DOUGOUDAI', 'GARDI 1', 'GARDI 2', 'GOULMOUN', 'KOKO', 'KOTO 1',
       'KOTO 2', 'MINAWA', 'VOUNDARAM', "WIDI VO'O", 'YIKA 1', 'YIKA 2',
       'DEGUE', 'GAIDA', 'GANGANG 1', 'GUIZEGUE 1', 'GUIZEGUE 2'

#### Nombre de ménages par quartier

In [12]:
population.QUARTIER.value_counts(ascending=True)

DIGUIMBE             5
DARAM                5
GOUNOURGOU           6
BODINGA              7
LANGUI               7
                    ..
DINGUE              46
TCHONG 2            47
MINAWA              47
HLAGAM GOURIVA 2    48
GUIA WIDI           53
Name: QUARTIER, Length: 82, dtype: int64

## Ajout de nouvelles  colonnes permettant de calculer les métriques.
- "Total" ou nombre total d'individus par menage, 
- "< 1" ou nombre total d'individus de moins d'un an,
- 'menage sans < 1' si un menage possède des enfants de moins d'un an ou pas. C'est une variable booléenne

In [13]:
# "Total" : nombre total d'individus dans le ménage
population['Total'] = population['< 1 MASC'] + population['1 - 4 MASC'] + population['5- 15 MASC'] + population['> 15 MASC'] + population['< 1 FEM'] + population['1 - 4 FEM'] + population['5 - 15 FEM'] + population['> 15 FEM'] 

In [14]:
# "< 1":Nombre d'enfants de moins d'un an dans le ménage
population['< 1'] = population['< 1 MASC'] + population['< 1 FEM']

In [15]:
# "menage sans < 1": le ménage a-t-il des enfants de moins d'un an ?. variable booléenne
population['menage sans < 1'] = population['< 1'] == 0

# Les différents type d'échantillonnage

## 1. Echantillonnage aléatoire simple.
Dans un échantillonnage aléatoire simple, chaque unité d’échantillonnage de la population a une chance égale d’être incluse dans l’échantillon. Pour mettre cette technique en œuvre, il faut d’abord dresser une liste de toutes les unités de la population observée.
2 modalités
- Réaliser un échantillon aléatoire simple de 300 ménages à partir des 1918 ménages
- Réaliser un échantillon aléatoire simple en collectant une fraction de la base de sondage (15 % par exemple)

### 1.1 Echantillonage aléatoire simple à nombre fixe (exemple 300 ménages choisis aléatoirement)

In [16]:
echantillon_aleatoire_simple = population.sample(300, random_state=2022)

In [17]:
echantillon_aleatoire_simple

,LAWANAT,QUARTIER,N° MENAGE,< 1 MASC,1 - 4 MASC,5- 15 MASC,> 15 MASC,< 1 FEM,1 - 4 FEM,5 - 15 FEM,> 15 FEM,Total,< 1,menage sans < 1
1685,WIDIGUE,HLAGAM GOURIVA 2,19,0,0,1,1,0,2,2,3,9,0,True
635,MERINGUE 1,BACKSARA,10,0,2,4,3,0,1,2,8,20,0,True
903,MERINGUE 2,NETONGAI,16,0,0,0,2,0,0,0,1,3,0,True
27,AGOLLA,BERE CENTRE,14,0,0,3,1,0,0,5,2,11,0,True
664,MERINGUE 1,DEKDEREK,21,0,0,2,1,0,0,3,2,8,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,AGOLLA,AGOLLA DOUNA,14,0,1,1,1,0,0,4,1,8,0,True
32,AGOLLA,BERE CENTRE,19,0,0,0,0,0,0,2,2,4,0,True
1916,WIDIGUE,ZENGUE,30,1,2,0,3,0,1,2,3,12,1,False
424,GUEMERE,KOUSSOUK,7,0,0,3,4,0,1,4,8,20,0,True


In [18]:
# Verification de la taille de l'échantillon
echantillon_aleatoire_simple.shape

(300, 14)

In [19]:
echantillon_aleatoire_simple.LAWANAT.unique()

array(['WIDIGUE', 'MERINGUE 1', 'MERINGUE 2', 'AGOLLA', 'WIDI',
       'MERINGUE 3', 'GUEMERE', 'GUIA'], dtype=object)

In [20]:
echantillon_aleatoire_simple.LAWANAT.value_counts()

WIDI          64
WIDIGUE       63
AGOLLA        52
MERINGUE 1    34
MERINGUE 2    34
GUEMERE       25
GUIA          18
MERINGUE 3    10
Name: LAWANAT, dtype: int64

### 1.2 Echantillonage aléatoire simple d'une fraction de la base de sondage (exemple 15 % ménages choisis aléatoirement)

In [21]:
fraction = 300/1918
fraction

0.15641293013555788

In [22]:
echantillon_aleatoire_simple_2 = population.sample(frac=0.15, random_state=2023)

In [23]:
echantillon_aleatoire_simple_2

,LAWANAT,QUARTIER,N° MENAGE,< 1 MASC,1 - 4 MASC,5- 15 MASC,> 15 MASC,< 1 FEM,1 - 4 FEM,5 - 15 FEM,> 15 FEM,Total,< 1,menage sans < 1
588,GUIA,GUIA WIDI,16,0,1,2,1,1,0,1,2,8,1,False
1041,MERINGUE 3,GOLONGOBOYE 1,6,0,1,0,1,0,1,1,1,5,0,True
877,MERINGUE 2,GANDJANG,18,0,1,3,1,0,0,3,2,10,0,True
149,AGOLLA,BOUROU 2,20,0,0,4,2,0,0,1,7,14,0,True
974,MERINGUE 2,TCHONG 1,30,0,1,4,4,0,0,6,3,18,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,MERINGUE 2,TCHONG 2,25,0,2,0,1,0,0,0,1,4,0,True
324,GUEMERE,BOURMI RAVANTONGAI,13,0,1,2,2,0,0,0,2,7,0,True
511,GUIA,GUIA MOKOLOM,9,1,2,1,3,0,1,1,2,11,1,False
866,MERINGUE 2,GANDJANG,7,0,0,1,0,0,0,1,2,4,0,True


In [24]:
echantillon_aleatoire_simple_2.LAWANAT.value_counts()

WIDI          61
WIDIGUE       55
AGOLLA        43
GUIA          30
GUEMERE       30
MERINGUE 2    29
MERINGUE 1    26
MERINGUE 3    14
Name: LAWANAT, dtype: int64

In [25]:
population.LAWANAT.value_counts(normalize=True)

WIDI          0.230970
WIDIGUE       0.186131
AGOLLA        0.142336
MERINGUE 2    0.106361
MERINGUE 1    0.100104
GUIA          0.096976
GUEMERE       0.087070
MERINGUE 3    0.050052
Name: LAWANAT, dtype: float64

In [26]:
echantillon_aleatoire_simple.LAWANAT.value_counts(normalize=True)

WIDI          0.213333
WIDIGUE       0.210000
AGOLLA        0.173333
MERINGUE 1    0.113333
MERINGUE 2    0.113333
GUEMERE       0.083333
GUIA          0.060000
MERINGUE 3    0.033333
Name: LAWANAT, dtype: float64

In [27]:
echantillon_aleatoire_simple_2.LAWANAT.value_counts(normalize=True)

WIDI          0.211806
WIDIGUE       0.190972
AGOLLA        0.149306
GUIA          0.104167
GUEMERE       0.104167
MERINGUE 2    0.100694
MERINGUE 1    0.090278
MERINGUE 3    0.048611
Name: LAWANAT, dtype: float64

#### Comparaison des différents échantillons (proportion de ménages)

In [28]:
df=pd.concat([population.LAWANAT.value_counts(normalize=True), echantillon_aleatoire_simple_2.LAWANAT.value_counts(normalize=True),echantillon_aleatoire_simple.LAWANAT.value_counts(normalize=True)],axis=1)

In [29]:
df.columns = ["Pop", "Echantillon_aleatoire_simple_2", "Echantillon_aleatoire_simple_1"]

In [30]:
df

,Pop,Echantillon_aleatoire_simple_2,Echantillon_aleatoire_simple_1
WIDI,0.230970,0.211806,0.213333
WIDIGUE,0.186131,0.190972,0.210000
AGOLLA,0.142336,0.149306,0.173333
MERINGUE 2,0.106361,0.100694,0.113333
MERINGUE 1,0.100104,0.090278,0.113333
GUIA,0.096976,0.104167,0.060000
GUEMERE,0.087070,0.104167,0.083333
MERINGUE 3,0.050052,0.048611,0.033333


#### Observation
On ne note pas de grandes variations dans les proportions de ménages entre les différents types d'échantillon.

## 2. Echantillonnage systématique.

L’échantillonnage systématique signifie qu’il existe un écart, ou un intervalle, ou pas de sondage, entre chaque unité sélectionnée dans l’échantillon. Pour éviter un biais dû à une base dde sondage qui aurait subit un tri, une des modalité serait de mélanger la base de sonfage avant le tirage.

### 2.1 Echantillonnage systématique sans mélange

Pour effectuer un échantillon systematique d'une taille de 300, il faut calculer la pas de sondage qui est de 1918/300. Avant de réaliser le tirage, nous pouvons aléatoirement choisir l'indice de la première unité de sondage entre 0 et le pas de sondage ôté de 1.

In [31]:
pas_de_sondage = 1918//300

In [32]:
pas_de_sondage

6

In [33]:
echantillon_systematique = population.iloc[::pas_de_sondage]

In [34]:
echantillon_systematique

,LAWANAT,QUARTIER,N° MENAGE,< 1 MASC,1 - 4 MASC,5- 15 MASC,> 15 MASC,< 1 FEM,1 - 4 FEM,5 - 15 FEM,> 15 FEM,Total,< 1,menage sans < 1
0,AGOLLA,AGOLLA DOUNA,1,0,1,1,3,0,2,3,4,14,0,True
6,AGOLLA,AGOLLA DOUNA,7,1,0,1,1,0,0,3,1,7,1,False
12,AGOLLA,AGOLLA DOUNA,13,0,1,1,2,0,0,5,1,10,0,True
18,AGOLLA,BERE CENTRE,5,0,0,1,1,0,0,2,2,6,0,True
24,AGOLLA,BERE CENTRE,11,0,0,0,1,0,0,0,2,3,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,WIDIGUE,ZENGUE,4,0,1,0,1,1,0,1,1,5,1,False
1896,WIDIGUE,ZENGUE,10,2,0,0,1,0,1,2,3,9,2,False
1902,WIDIGUE,ZENGUE,16,0,2,1,1,0,1,4,3,12,0,True
1908,WIDIGUE,ZENGUE,22,0,0,0,1,0,0,0,3,4,0,True


In [35]:
echantillon_systematique.LAWANAT.value_counts(normalize=True)

WIDI          0.231250
WIDIGUE       0.184375
AGOLLA        0.143750
MERINGUE 2    0.106250
MERINGUE 1    0.100000
GUIA          0.096875
GUEMERE       0.087500
MERINGUE 3    0.050000
Name: LAWANAT, dtype: float64

L'échantillonnage systématique pose un problème : si les données ont été triées, ou s'il existe une sorte de modèle ou de signification derrière l'ordre des lignes, l'échantillon résultant peut ne pas être représentatif de l'ensemble de la population. Le problème peut être résolu en mélangeant les lignes, mais alors l'échantillonnage systématique équivaut à un échantillonnage aléatoire simple.

### 2.2 Echantillonnage systématique précédé d'un mélange

In [36]:
# Ajouter une colonne index à la population
population_id = population.reset_index(drop=True)

In [37]:
population_id.head()

,LAWANAT,QUARTIER,N° MENAGE,< 1 MASC,1 - 4 MASC,5- 15 MASC,> 15 MASC,< 1 FEM,1 - 4 FEM,5 - 15 FEM,> 15 FEM,Total,< 1,menage sans < 1
0,AGOLLA,AGOLLA DOUNA,1,0,1,1,3,0,2,3,4,14,0,True
1,AGOLLA,AGOLLA DOUNA,2,0,1,2,2,0,0,1,2,8,0,True
2,AGOLLA,AGOLLA DOUNA,3,0,1,3,1,0,1,0,3,9,0,True
3,AGOLLA,AGOLLA DOUNA,4,0,0,1,3,1,0,0,1,6,1,False
4,AGOLLA,AGOLLA DOUNA,5,0,0,2,2,0,1,3,2,10,0,True


In [38]:
# Melanger les lignes de la population
population_melangee = population_id.sample(frac=1, random_state=2024)

In [39]:
population_melangee.head()

,LAWANAT,QUARTIER,N° MENAGE,< 1 MASC,1 - 4 MASC,5- 15 MASC,> 15 MASC,< 1 FEM,1 - 4 FEM,5 - 15 FEM,> 15 FEM,Total,< 1,menage sans < 1
561,GUIA,GUIA RAKAYE,18,0,1,0,0,0,1,0,1,3,0,True
32,AGOLLA,BERE CENTRE,19,0,0,0,0,0,0,2,2,4,0,True
918,MERINGUE 2,SOUALAYE,3,1,0,2,4,0,2,1,4,14,1,False
1789,WIDIGUE,VALA,10,0,1,3,3,0,3,5,4,19,0,True
301,GUEMERE,BOURMI NGUEVELEKE,29,0,0,0,1,0,0,0,2,3,0,True


In [40]:
# Supprimer la colonne index
population_melangee = population_melangee.reset_index(drop=True)

In [41]:
population_melangee.head()

,LAWANAT,QUARTIER,N° MENAGE,< 1 MASC,1 - 4 MASC,5- 15 MASC,> 15 MASC,< 1 FEM,1 - 4 FEM,5 - 15 FEM,> 15 FEM,Total,< 1,menage sans < 1
0,GUIA,GUIA RAKAYE,18,0,1,0,0,0,1,0,1,3,0,True
1,AGOLLA,BERE CENTRE,19,0,0,0,0,0,0,2,2,4,0,True
2,MERINGUE 2,SOUALAYE,3,1,0,2,4,0,2,1,4,14,1,False
3,WIDIGUE,VALA,10,0,1,3,3,0,3,5,4,19,0,True
4,GUEMERE,BOURMI NGUEVELEKE,29,0,0,0,1,0,0,0,2,3,0,True


In [42]:
population_melangee.reset_index()

,index,LAWANAT,QUARTIER,N° MENAGE,< 1 MASC,1 - 4 MASC,5- 15 MASC,> 15 MASC,< 1 FEM,1 - 4 FEM,5 - 15 FEM,> 15 FEM,Total,< 1,menage sans < 1
0,0,GUIA,GUIA RAKAYE,18,0,1,0,0,0,1,0,1,3,0,True
1,1,AGOLLA,BERE CENTRE,19,0,0,0,0,0,0,2,2,4,0,True
2,2,MERINGUE 2,SOUALAYE,3,1,0,2,4,0,2,1,4,14,1,False
3,3,WIDIGUE,VALA,10,0,1,3,3,0,3,5,4,19,0,True
4,4,GUEMERE,BOURMI NGUEVELEKE,29,0,0,0,1,0,0,0,2,3,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1913,1913,GUIA,GUIA MOKOLOM,37,0,0,0,1,0,0,4,2,7,0,True
1914,1914,MERINGUE 1,DARAM,2,0,2,3,4,1,0,4,6,20,1,False
1915,1915,GUIA,GUIA WIDI,36,0,3,4,3,0,0,3,4,17,0,True
1916,1916,WIDI,YIKA 1,28,0,1,1,4,1,4,4,5,20,1,False


In [43]:
echantillon_systematique_2 = population_melangee.iloc[::pas_de_sondage]

In [44]:
echantillon_systematique_2.head()

,LAWANAT,QUARTIER,N° MENAGE,< 1 MASC,1 - 4 MASC,5- 15 MASC,> 15 MASC,< 1 FEM,1 - 4 FEM,5 - 15 FEM,> 15 FEM,Total,< 1,menage sans < 1
0,GUIA,GUIA RAKAYE,18,0,1,0,0,0,1,0,1,3,0,True
6,WIDI,DINGUE,28,0,2,0,1,0,0,0,4,7,0,True
12,WIDIGUE,ZENGUE,1,0,1,0,1,0,0,1,2,5,0,True
18,WIDIGUE,HLAGAM GOURIVA 3,4,0,0,1,1,0,2,2,2,8,0,True
24,WIDI,MINAWA,44,0,0,1,3,0,0,0,2,6,0,True


In [45]:
echantillon_systematique_2.shape

(320, 14)

In [46]:
echantillon_systematique_2.LAWANAT.value_counts(normalize=True)

WIDI          0.221875
WIDIGUE       0.187500
AGOLLA        0.128125
MERINGUE 1    0.125000
MERINGUE 2    0.109375
GUIA          0.103125
GUEMERE       0.065625
MERINGUE 3    0.059375
Name: LAWANAT, dtype: float64

#### Comparaison des différents échantillon en terme de proportion de ménage ou unité de sondage

In [47]:
df2=pd.concat([population.LAWANAT.value_counts(normalize=True), echantillon_systematique_2.LAWANAT.value_counts(normalize=True),echantillon_systematique.LAWANAT.value_counts(normalize=True)],axis=1)

In [48]:
df2.columns = ["Pop", "Sampl_systematic_2", "Sampl_systematic_1"]

In [49]:
df2

,Pop,Sampl_systematic_2,Sampl_systematic_1
WIDI,0.230970,0.221875,0.231250
WIDIGUE,0.186131,0.187500,0.184375
AGOLLA,0.142336,0.128125,0.143750
MERINGUE 2,0.106361,0.109375,0.106250
MERINGUE 1,0.100104,0.125000,0.100000
GUIA,0.096976,0.103125,0.096875
GUEMERE,0.087070,0.065625,0.087500
MERINGUE 3,0.050052,0.059375,0.050000


#### Observation
On ne note pas de différence entre les différents types d'échantillons

## 3. Echantillonnage stratifié

Cette méthode permet de représenter les sous- groupes d’une population hétérogène. On peut répartir l’échantillon de façon proportionnelle à la taille des sous-groupes et ainsi tous les individus possèdent une même chance d’être sélectionnés comme pour l’échantillon aléatoire simple. On peut sélectionner un même nombre d'unité à l'intérieur de chaque strate.

## 3.1 Echantillonnage stratifié proportionnel.

Echantillon stratifié (strate = LAWANAT). 20 % par strate

In [50]:
echantillon_strat_prop = population.groupby('LAWANAT').sample(frac=0.2, random_state=2025)

In [51]:
echantillon_strat_prop

,LAWANAT,QUARTIER,N° MENAGE,< 1 MASC,1 - 4 MASC,5- 15 MASC,> 15 MASC,< 1 FEM,1 - 4 FEM,5 - 15 FEM,> 15 FEM,Total,< 1,menage sans < 1
0,AGOLLA,AGOLLA DOUNA,1,0,1,1,3,0,2,3,4,14,0,True
175,AGOLLA,DANAY 1,21,0,1,0,2,1,0,0,3,7,1,False
56,AGOLLA,BIDWERE 1,19,0,2,4,5,0,1,3,5,20,0,True
133,AGOLLA,BOUROU 2,4,1,1,0,1,0,0,0,1,4,1,False
215,AGOLLA,KOMKOSSAYE 1,17,1,0,0,4,1,1,2,4,13,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707,WIDIGUE,HLAGAM GOURIVA 2,41,1,3,1,0,0,0,2,1,8,1,False
1823,WIDIGUE,VEGELENDA,26,0,0,1,1,0,1,1,1,5,0,True
1709,WIDIGUE,HLAGAM GOURIVA 2,43,0,0,0,1,0,1,1,2,5,0,True
1743,WIDIGUE,HLAGAM ZIGENDJI,17,0,0,0,0,0,0,3,1,4,0,True


In [52]:
echantillon_strat_prop.LAWANAT.value_counts()

WIDI          89
WIDIGUE       71
AGOLLA        55
MERINGUE 2    41
MERINGUE 1    38
GUIA          37
GUEMERE       33
MERINGUE 3    19
Name: LAWANAT, dtype: int64

In [53]:
echantillon_strat_prop.LAWANAT.value_counts(normalize=True)

WIDI          0.232376
WIDIGUE       0.185379
AGOLLA        0.143603
MERINGUE 2    0.107050
MERINGUE 1    0.099217
GUIA          0.096606
GUEMERE       0.086162
MERINGUE 3    0.049608
Name: LAWANAT, dtype: float64

In [54]:
echantillon_strat_prop.LAWANAT.value_counts()/population.LAWANAT.value_counts()

WIDI          0.200903
WIDIGUE       0.198880
AGOLLA        0.201465
MERINGUE 2    0.200980
MERINGUE 1    0.197917
GUIA          0.198925
GUEMERE       0.197605
MERINGUE 3    0.197917
Name: LAWANAT, dtype: float64

## 3.2 Echantillonnage stratifié à nombre égal.

In [55]:
population.LAWANAT.value_counts()

WIDI          443
WIDIGUE       357
AGOLLA        273
MERINGUE 2    204
MERINGUE 1    192
GUIA          186
GUEMERE       167
MERINGUE 3     96
Name: LAWANAT, dtype: int64

In [56]:
echantillon_strat_eq_nb = population.groupby('LAWANAT').sample(n=20, random_state=2026)

In [57]:
echantillon_strat_eq_nb

,LAWANAT,QUARTIER,N° MENAGE,< 1 MASC,1 - 4 MASC,5- 15 MASC,> 15 MASC,< 1 FEM,1 - 4 FEM,5 - 15 FEM,> 15 FEM,Total,< 1,menage sans < 1
222,AGOLLA,KOMKOSSAYE 1,24,0,0,1,3,0,0,2,1,7,0,True
255,AGOLLA,KOMKOSSAYE 2,12,0,1,0,5,0,0,2,2,10,0,True
53,AGOLLA,BIDWERE 1,16,0,0,2,2,1,0,1,1,7,1,False
267,AGOLLA,MAGA,8,0,0,1,2,0,1,2,1,7,0,True
128,AGOLLA,BOUROU 1,9,0,2,1,2,1,1,1,4,12,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1888,WIDIGUE,ZENGUE,2,0,2,0,1,0,0,1,1,5,0,True
1599,WIDIGUE,GANGANG 1,5,0,2,0,1,0,0,1,2,6,0,True
1753,WIDIGUE,HLAGAM ZIGENDJI,27,0,2,2,2,0,0,1,2,9,0,True
1716,WIDIGUE,HLAGAM GOURIVA 3,2,0,2,2,1,0,0,3,1,9,0,True


In [58]:
echantillon_strat_eq_nb.LAWANAT.value_counts()

AGOLLA        20
GUEMERE       20
GUIA          20
MERINGUE 1    20
MERINGUE 2    20
MERINGUE 3    20
WIDI          20
WIDIGUE       20
Name: LAWANAT, dtype: int64

#### Comparaison population vs différents échantillons stratifié

In [59]:
df3=pd.concat([population.LAWANAT.value_counts(normalize=True), echantillon_strat_prop.LAWANAT.value_counts(normalize=True),echantillon_strat_eq_nb.LAWANAT.value_counts(normalize=True)],axis=1)

In [60]:
df3.columns = ['Population', 'Stratifié proportionnel', 'Stratifié à nombre égal']

In [61]:
df3

,Population,Stratifié proportionnel,Stratifié à nombre égal
WIDI,0.230970,0.232376,0.125
WIDIGUE,0.186131,0.185379,0.125
AGOLLA,0.142336,0.143603,0.125
MERINGUE 2,0.106361,0.107050,0.125
MERINGUE 1,0.100104,0.099217,0.125
GUIA,0.096976,0.096606,0.125
GUEMERE,0.087070,0.086162,0.125
MERINGUE 3,0.050052,0.049608,0.125


#### Observation
On note que l'échantillon stratifié à nombre égal présente la même fraction de ménages choisis dans chaque lawanat. Les proportions entre la population et l'échantillon stratifié proportionnel sont quasi identiques

## 4. Echantillonnage pondéré.

Dans le contexte d’un échantillonnage probabiliste, il faut que chaque unité de la population observée ait une probabilité connue d’être incluse dans l’échantillon, mais il n’est pas nécessaire qu’elle soit la même pour tous. Si la base de sondage renferme de l’information sur la taille de chaque unité (comme le nombre de ménages de chacun des lawanats qui y sont inscrites) et si la taille de ces unités varie, on peut utiliser cette information dans le cadre de la sélection de l’échantillon afin d’en accroître l’efficacité.

Dans ce type d’échantillonnage les unités n’ont pas la même probabilité d’apparition dans l’échantillon final. Il leur est affecté un facteur de pondération. Les unités appartenant au même lawanat se sont vues attribué le même poids. La valeur du poids est le nombre de ménages de la base de sondage divisé par le nombre de ménage dans le lawanat d’appartenance. Nous avons choisi d’augmenter la probabilité d’apparition des ménages de strates (LAWANAT) sous représentées. Une colonne "weigth a ensuite été créée pour cette pondération.

#### Création de la colonne "weight" pour la pondération
Calcul des poids pour chaque ménage; weight = nombre de menage total / nombre de menages du lawanat

In [62]:
population_weight = round(population.shape[0]/population.LAWANAT.value_counts(),0)

In [63]:
population_weight

WIDI           4.0
WIDIGUE        5.0
AGOLLA         7.0
MERINGUE 2     9.0
MERINGUE 1    10.0
GUIA          10.0
GUEMERE       11.0
MERINGUE 3    20.0
Name: LAWANAT, dtype: float64

#### Le lawanat MERINGUE qui a la plus petite quantité de ménage a le plus grand poids (20). A l'inverse le plus grand Lawanat WIDI a le plus faible poids (4)

In [64]:
condition_list = list(population_weight.index.values)

In [65]:
condition_list

['WIDI',
 'WIDIGUE',
 'AGOLLA',
 'MERINGUE 2',
 'MERINGUE 1',
 'GUIA',
 'GUEMERE',
 'MERINGUE 3']

In [66]:
values = population_weight.to_list()

In [67]:
values

[4.0, 5.0, 7.0, 9.0, 10.0, 10.0, 11.0, 20.0]

In [68]:
conditions = [
  population.LAWANAT == "WIDI",
    population.LAWANAT == "WIDIGUE",
    population.LAWANAT == "AGOLLA",
    population.LAWANAT == "MERINGUE 2",
    population.LAWANAT == "MERINGUE 1",
    population.LAWANAT == "GUIA",
    population.LAWANAT == "GUEMERE",
    population.LAWANAT == "MERINGUE 3"
]

In [69]:
population["weight"] = np.select(conditions, values, default=0)

In [70]:
population.weight.head()

0    7.0
1    7.0
2    7.0
3    7.0
4    7.0
Name: weight, dtype: float64

In [71]:
echantillon_pondere = population.sample(n=300, weights="weight", random_state=2027)

In [72]:
echantillon_pondere.head()

,LAWANAT,QUARTIER,N° MENAGE,< 1 MASC,1 - 4 MASC,5- 15 MASC,> 15 MASC,< 1 FEM,1 - 4 FEM,5 - 15 FEM,> 15 FEM,Total,< 1,menage sans < 1,weight
1271,WIDI,GARDI 2,9,0,1,3,1,0,1,0,1,7,0,True,4.0
1632,WIDIGUE,GUIZEGUE 1,13,0,0,2,5,0,2,2,3,14,0,True,5.0
1193,WIDI,DJALWA,30,0,2,3,3,1,0,0,2,11,1,False,4.0
1543,WIDI,YIKA 2,11,0,0,0,2,0,1,0,3,6,0,True,4.0
861,MERINGUE 2,GANDJANG,2,0,2,0,2,0,1,3,5,13,0,True,9.0


In [73]:
echantillon_pondere.LAWANAT.value_counts()

WIDI          42
WIDIGUE       42
GUEMERE       40
MERINGUE 2    39
AGOLLA        39
MERINGUE 1    36
GUIA          32
MERINGUE 3    30
Name: LAWANAT, dtype: int64

In [74]:
echantillon_pondere.LAWANAT.value_counts(normalize=True)

WIDI          0.140000
WIDIGUE       0.140000
GUEMERE       0.133333
MERINGUE 2    0.130000
AGOLLA        0.130000
MERINGUE 1    0.120000
GUIA          0.106667
MERINGUE 3    0.100000
Name: LAWANAT, dtype: float64

In [75]:
population.LAWANAT.value_counts(normalize=True)

WIDI          0.230970
WIDIGUE       0.186131
AGOLLA        0.142336
MERINGUE 2    0.106361
MERINGUE 1    0.100104
GUIA          0.096976
GUEMERE       0.087070
MERINGUE 3    0.050052
Name: LAWANAT, dtype: float64

In [76]:
df4=pd.concat([population.LAWANAT.value_counts(normalize=True), echantillon_pondere.LAWANAT.value_counts(normalize=True)],axis=1)

In [77]:
df4.columns = ['Population', 'Pondéré']

In [78]:
df4

,Population,Pondéré
WIDI,0.230970,0.140000
WIDIGUE,0.186131,0.140000
AGOLLA,0.142336,0.130000
MERINGUE 2,0.106361,0.130000
MERINGUE 1,0.100104,0.120000
GUIA,0.096976,0.106667
GUEMERE,0.087070,0.133333
MERINGUE 3,0.050052,0.100000


#### Observation sur l'échantillonnage pondéré.
On constate un réequilibrage des échantillons au niveau des lawanat avec un sous-echantillonnage des lawanats avec un grand nombre de ménages et une sur-echantillonnage des lawanats avec un petit nombre de ménages.

## 5. Echantillonnage en grappe.

Nous allons réaliser un échantillon en grappe (une grappe étant un quartier).Nous allons choisir aléatoirement un certain nombre de quartiers (grappes) quartiers de l’ensemble des quartiers de la base de sondage. 10 grappes seront aléatoirement choisies dans l'ensemble de 82 quartiers (grappes). Ensuite toutes les unités des grappes choisies sont insérées dans l'échantillon. L’échantillonnage en grappe constitue très souvent le premier degré d’un échantillonnage à plusieurs degrés. Le second degré sera un échantillonnage aléatoire simple (3 ménages dans chaqune des grappes) dans notre exemple obtenues de l'échantillon obtenue au premier degré.

In [79]:
liste_quartiers = list(population.QUARTIER.unique())

In [80]:
liste_quartiers

['AGOLLA DOUNA',
 'BERE CENTRE',
 'BIDWERE 1',
 'BIDWERE 2',
 'BIDWERE 3',
 'BODINGA',
 'BOUROU 1',
 'BOUROU 2',
 'DANAY 1',
 'DANAY 2',
 'KOMKOSSAYE 1',
 'KOMKOSSAYE 2',
 'MAGA',
 'BOURMI NGUEVELEKE',
 'BOURMI RAVANTONGAI',
 'DAGADAGA',
 'DANAY DOUVOUGOUNI',
 'DANAY VOULHANA',
 'GUEMERE CENTRE',
 'HIRAY',
 'KOUSSOUK',
 'YOUNA',
 'GUIA BACKSARA',
 'GUIA BIDWERE',
 'GUIA MOKOLOM',
 'GUIA RAKAYE',
 'GUIA WIDI',
 'BACKSARA',
 'DARAM',
 'DEKDEREK',
 'DIGUIMBE',
 'DJO',
 'GOHO',
 'LANGUI',
 'MARANDJAM',
 'MOKO',
 'NOULTONGAYE CENTRE',
 'SAKATONGAI',
 'FITIMLAYE 1',
 'FITIMLAYE 2',
 'GANDJANG',
 'NETONGAI',
 'SOUALAYE',
 'TCHONG 1',
 'TCHONG 2',
 'BANGAL',
 'GOLONGOBOYE 1',
 'GOLONGOBOYE 2',
 'GOUNOURGOU',
 'MASSOUANG 1',
 'MOUNHA ',
 'DINGUE',
 'DJALWA',
 'DOUGOUDAI',
 'GARDI 1',
 'GARDI 2',
 'GOULMOUN',
 'KOKO',
 'KOTO 1',
 'KOTO 2',
 'MINAWA',
 'VOUNDARAM',
 "WIDI VO'O",
 'YIKA 1',
 'YIKA 2',
 'DEGUE',
 'GAIDA',
 'GANGANG 1',
 'GUIZEGUE 1',
 'GUIZEGUE 2',
 'HLAGAM GOURIVA 1',
 'HLAGAM GOU

In [81]:
len(liste_quartiers)

82

Il existe 82 quartiers dans cette base de sondage. Nous voulons selectionner 10 quartiers (10 grappes). Cette étape qui est un échantillonnage en grappe et toutes les unités (ménages) seront inclues dans l'échantillon.Il peut aussi être considéré comme un échantillonnage de premier degré. Dans ce dernier cas un autre échantillonnage à lieu sur les grappes choisies.

In [82]:
import random

In [83]:
random.seed(2027)
echantillon_quartiers = random.sample(liste_quartiers, k=10)

In [84]:
echantillon_quartiers

['BOURMI NGUEVELEKE',
 'KOTO 2',
 'KOMKOSSAYE 1',
 'BERE CENTRE',
 'NETONGAI',
 'DINGUE',
 'GUIA BIDWERE',
 'VALA',
 'KAMAS 2',
 'TCHONG 2']

In [85]:
population.columns

Index(['LAWANAT', 'QUARTIER', 'N° MENAGE', '< 1 MASC', '1 - 4 MASC',
       '5- 15 MASC', '> 15 MASC', '< 1 FEM', '1 - 4 FEM', '5 - 15 FEM',
       '> 15 FEM', 'Total', '< 1', 'menage sans < 1', 'weight'],
      dtype='object')

In [86]:
population.head(2)

,LAWANAT,QUARTIER,N° MENAGE,< 1 MASC,1 - 4 MASC,5- 15 MASC,> 15 MASC,< 1 FEM,1 - 4 FEM,5 - 15 FEM,> 15 FEM,Total,< 1,menage sans < 1,weight
0,AGOLLA,AGOLLA DOUNA,1,0,1,1,3,0,2,3,4,14,0,True,7.0
1,AGOLLA,AGOLLA DOUNA,2,0,1,2,2,0,0,1,2,8,0,True,7.0


In [87]:
population

,LAWANAT,QUARTIER,N° MENAGE,< 1 MASC,1 - 4 MASC,5- 15 MASC,> 15 MASC,< 1 FEM,1 - 4 FEM,5 - 15 FEM,> 15 FEM,Total,< 1,menage sans < 1,weight
0,AGOLLA,AGOLLA DOUNA,1,0,1,1,3,0,2,3,4,14,0,True,7.0
1,AGOLLA,AGOLLA DOUNA,2,0,1,2,2,0,0,1,2,8,0,True,7.0
2,AGOLLA,AGOLLA DOUNA,3,0,1,3,1,0,1,0,3,9,0,True,7.0
3,AGOLLA,AGOLLA DOUNA,4,0,0,1,3,1,0,0,1,6,1,False,7.0
4,AGOLLA,AGOLLA DOUNA,5,0,0,2,2,0,1,3,2,10,0,True,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1913,WIDIGUE,ZENGUE,27,1,2,3,4,0,1,1,2,14,1,False,5.0
1914,WIDIGUE,ZENGUE,28,2,1,2,5,0,0,1,5,16,2,False,5.0
1915,WIDIGUE,ZENGUE,29,1,0,3,3,1,0,6,3,17,2,False,5.0
1916,WIDIGUE,ZENGUE,30,1,2,0,3,0,1,2,3,12,1,False,5.0


In [88]:
condition = population['QUARTIER'].isin(echantillon_quartiers)

In [89]:
quartiers_filtres = population[condition]

In [90]:
quartiers_filtres['QUARTIER'].value_counts()

TCHONG 2             47
DINGUE               46
KOMKOSSAYE 1         45
BOURMI NGUEVELEKE    39
GUIA BIDWERE         35
NETONGAI             28
KOTO 2               26
BERE CENTRE          24
VALA                 18
KAMAS 2               8
Name: QUARTIER, dtype: int64

In [91]:
quartiers_filtres.head()

,LAWANAT,QUARTIER,N° MENAGE,< 1 MASC,1 - 4 MASC,5- 15 MASC,> 15 MASC,< 1 FEM,1 - 4 FEM,5 - 15 FEM,> 15 FEM,Total,< 1,menage sans < 1,weight
14,AGOLLA,BERE CENTRE,1,0,2,3,1,0,0,2,2,10,0,True,7.0
15,AGOLLA,BERE CENTRE,2,0,0,0,2,0,1,3,1,7,0,True,7.0
16,AGOLLA,BERE CENTRE,3,0,0,0,1,0,1,0,2,4,0,True,7.0
17,AGOLLA,BERE CENTRE,4,0,1,0,1,1,0,0,1,4,1,False,7.0
18,AGOLLA,BERE CENTRE,5,0,0,1,1,0,0,2,2,6,0,True,7.0


In [92]:
# Si la colonne était de type "categorie", on allait executer d'abord cette ligne de code ci dessous pour enlever les categories sans lignes
# quartiers_filtres['QUARTIER'] = quartiers_filtres['QUARTIER'].cat.remove_unused_categories()

#### Second degré de l'échantillonnage

In [93]:
echantillon_en_grappe = quartiers_filtres.groupby('QUARTIER').sample(n=3, random_state=2028)

In [94]:
echantillon_en_grappe.QUARTIER.value_counts()

BERE CENTRE          3
BOURMI NGUEVELEKE    3
DINGUE               3
GUIA BIDWERE         3
KAMAS 2              3
KOMKOSSAYE 1         3
KOTO 2               3
NETONGAI             3
TCHONG 2             3
VALA                 3
Name: QUARTIER, dtype: int64

In [95]:
echantillon_en_grappe

,LAWANAT,QUARTIER,N° MENAGE,< 1 MASC,1 - 4 MASC,5- 15 MASC,> 15 MASC,< 1 FEM,1 - 4 FEM,5 - 15 FEM,> 15 FEM,Total,< 1,menage sans < 1,weight
29,AGOLLA,BERE CENTRE,16,0,1,0,3,0,0,0,1,5,0,True,7.0
18,AGOLLA,BERE CENTRE,5,0,0,1,1,0,0,2,2,6,0,True,7.0
15,AGOLLA,BERE CENTRE,2,0,0,0,2,0,1,3,1,7,0,True,7.0
307,GUEMERE,BOURMI NGUEVELEKE,35,0,1,2,1,0,1,1,1,7,0,True,11.0
294,GUEMERE,BOURMI NGUEVELEKE,22,0,0,1,0,0,0,0,0,1,0,True,11.0
278,GUEMERE,BOURMI NGUEVELEKE,6,0,0,0,1,0,2,0,1,4,0,True,11.0
1140,WIDI,DINGUE,23,0,0,3,3,0,1,2,2,11,0,True,4.0
1149,WIDI,DINGUE,32,0,0,1,2,0,0,2,1,6,0,True,4.0
1148,WIDI,DINGUE,31,0,1,1,3,0,0,0,2,7,0,True,4.0
484,GUIA,GUIA BIDWERE,17,1,0,1,0,0,0,2,2,6,1,False,10.0


In [96]:
echantillon_en_grappe.shape

(30, 15)

# 6. Les métriques

   ## Calcul de 3 métriques au sein de la population ain
- Taille moyenne des ménages
- Proportion de ménages sans enfant de moins d'un an
- Proportion des moins d'un an par rapport à la population totale

## 6.1 Taille moyenne des ménages

#### Au sein de la population

In [97]:
population.Total.mean()

9.531282586027112

In [98]:
population.groupby('LAWANAT')['Total'].mean()

LAWANAT
AGOLLA         9.388278
GUEMERE        8.041916
GUIA           8.306452
MERINGUE 1    10.609375
MERINGUE 2     9.529412
MERINGUE 3     7.302083
WIDI          10.250564
WIDIGUE       10.103641
Name: Total, dtype: float64

#### Au sein des échantillons

In [99]:
echantillon_aleatoire_simple.Total.mean()

9.94

In [100]:
echantillon_aleatoire_simple.groupby('LAWANAT')['Total'].mean()

LAWANAT
AGOLLA        10.019231
GUEMERE        9.280000
GUIA           7.833333
MERINGUE 1    11.000000
MERINGUE 2     9.823529
MERINGUE 3     7.000000
WIDI          11.109375
WIDIGUE        9.507937
Name: Total, dtype: float64

In [101]:
echantillon_strat_prop.Total.mean()

9.240208877284596

In [102]:
echantillon_strat_prop.groupby('LAWANAT')['Total'].mean()

LAWANAT
AGOLLA         8.963636
GUEMERE        6.878788
GUIA           7.459459
MERINGUE 1    12.078947
MERINGUE 2     9.219512
MERINGUE 3     7.947368
WIDI           9.539326
WIDIGUE        9.943662
Name: Total, dtype: float64

In [103]:
echantillon_pondere.Total.mean()

9.01

In [104]:
echantillon_pondere.groupby('LAWANAT')['Total'].mean()

LAWANAT
AGOLLA         9.282051
GUEMERE        8.825000
GUIA           7.000000
MERINGUE 1    10.194444
MERINGUE 2     8.948718
MERINGUE 3     6.666667
WIDI           9.976190
WIDIGUE       10.214286
Name: Total, dtype: float64

In [105]:
echantillon_en_grappe.Total.mean()

9.5

In [106]:
echantillon_en_grappe.groupby('LAWANAT')['Total'].mean()

LAWANAT
AGOLLA         9.833333
GUEMERE        4.000000
GUIA           7.666667
MERINGUE 2     8.833333
WIDI           9.000000
WIDIGUE       14.000000
Name: Total, dtype: float64

## 6.2 Proportion de ménages sans enfant de moins d'un an

#### Au sein de la population (base de sondage)

In [107]:
population['menage sans < 1'].mean()

0.7278415015641293

In [108]:
population.groupby('LAWANAT')['menage sans < 1'].mean()

LAWANAT
AGOLLA        0.750916
GUEMERE       0.766467
GUIA          0.758065
MERINGUE 1    0.671875
MERINGUE 2    0.735294
MERINGUE 3    0.593750
WIDI          0.690745
WIDIGUE       0.784314
Name: menage sans < 1, dtype: float64

#### Au sein des différents échantillons

In [109]:
echantillon_aleatoire_simple['menage sans < 1'].mean()

0.7233333333333334

In [110]:
echantillon_aleatoire_simple.groupby('LAWANAT')['menage sans < 1'].mean()

LAWANAT
AGOLLA        0.750000
GUEMERE       0.640000
GUIA          0.777778
MERINGUE 1    0.705882
MERINGUE 2    0.764706
MERINGUE 3    0.600000
WIDI          0.625000
WIDIGUE       0.825397
Name: menage sans < 1, dtype: float64

In [111]:
echantillon_aleatoire_simple_2['menage sans < 1'].mean()

0.7569444444444444

In [112]:
echantillon_aleatoire_simple_2.groupby('LAWANAT')['menage sans < 1'].mean()

LAWANAT
AGOLLA        0.837209
GUEMERE       0.766667
GUIA          0.733333
MERINGUE 1    0.653846
MERINGUE 2    0.793103
MERINGUE 3    0.571429
WIDI          0.704918
WIDIGUE       0.836364
Name: menage sans < 1, dtype: float64

In [113]:
echantillon_systematique['menage sans < 1'].mean()

0.73125

In [114]:
echantillon_systematique.groupby('LAWANAT')['menage sans < 1'].mean()

LAWANAT
AGOLLA        0.739130
GUEMERE       0.750000
GUIA          0.709677
MERINGUE 1    0.625000
MERINGUE 2    0.705882
MERINGUE 3    0.750000
WIDI          0.702703
WIDIGUE       0.830508
Name: menage sans < 1, dtype: float64

In [115]:
echantillon_systematique_2['menage sans < 1'].mean()

0.734375

In [116]:
echantillon_systematique_2.groupby('LAWANAT')['menage sans < 1'].mean()

LAWANAT
AGOLLA        0.780488
GUEMERE       0.857143
GUIA          0.666667
MERINGUE 1    0.725000
MERINGUE 2    0.800000
MERINGUE 3    0.684211
WIDI          0.647887
WIDIGUE       0.783333
Name: menage sans < 1, dtype: float64

In [117]:
echantillon_strat_prop['menage sans < 1'].mean()

0.7310704960835509

In [118]:
echantillon_strat_prop.groupby('LAWANAT')['menage sans < 1'].mean()

LAWANAT
AGOLLA        0.763636
GUEMERE       0.848485
GUIA          0.783784
MERINGUE 1    0.500000
MERINGUE 2    0.829268
MERINGUE 3    0.526316
WIDI          0.707865
WIDIGUE       0.774648
Name: menage sans < 1, dtype: float64

In [119]:
echantillon_strat_eq_nb['menage sans < 1'].mean()

0.73125

In [120]:
echantillon_strat_eq_nb.groupby('LAWANAT')['menage sans < 1'].mean()

LAWANAT
AGOLLA        0.75
GUEMERE       0.70
GUIA          0.70
MERINGUE 1    0.80
MERINGUE 2    0.75
MERINGUE 3    0.75
WIDI          0.55
WIDIGUE       0.85
Name: menage sans < 1, dtype: float64

In [121]:
echantillon_pondere['menage sans < 1'].mean()

0.7366666666666667

In [122]:
echantillon_pondere.groupby('LAWANAT')['menage sans < 1'].mean()

LAWANAT
AGOLLA        0.641026
GUEMERE       0.775000
GUIA          0.781250
MERINGUE 1    0.694444
MERINGUE 2    0.871795
MERINGUE 3    0.666667
WIDI          0.690476
WIDIGUE       0.761905
Name: menage sans < 1, dtype: float64

In [123]:
echantillon_en_grappe['menage sans < 1'].mean()

0.8666666666666667

In [124]:
echantillon_en_grappe.groupby('LAWANAT')['menage sans < 1'].mean()

LAWANAT
AGOLLA        0.833333
GUEMERE       1.000000
GUIA          0.333333
MERINGUE 2    1.000000
WIDI          0.833333
WIDIGUE       1.000000
Name: menage sans < 1, dtype: float64

## 6.3 Proportion des moins d'un an par rapport à la population totale

#### Au sein de la population

In [125]:
round(population['< 1'].sum()/population['Total'].sum()*100,2)

3.54

In [126]:
round(population.groupby('LAWANAT')['< 1'].sum()/population.groupby('LAWANAT')['Total'].sum()*100,2)

LAWANAT
AGOLLA        3.00
GUEMERE       3.65
GUIA          3.43
MERINGUE 1    3.53
MERINGUE 2    3.45
MERINGUE 3    8.42
WIDI          3.83
WIDIGUE       2.66
dtype: float64

#### Au sein des échantillons

In [127]:
round(echantillon_aleatoire_simple['< 1'].sum()/echantillon_aleatoire_simple['Total'].sum()*100,2)

3.25

In [128]:
round(echantillon_aleatoire_simple.groupby('LAWANAT')['< 1'].sum()/echantillon_aleatoire_simple.groupby('LAWANAT')['Total'].sum()*100,2)

LAWANAT
AGOLLA         2.69
GUEMERE        4.31
GUIA           2.84
MERINGUE 1     3.48
MERINGUE 2     2.40
MERINGUE 3    12.86
WIDI           3.94
WIDIGUE        1.84
dtype: float64

In [129]:
round(echantillon_strat_prop['< 1'].sum()/echantillon_strat_prop['Total'].sum()*100,2)

3.62

In [130]:
round(echantillon_strat_prop.groupby('LAWANAT')['< 1'].sum()/echantillon_strat_prop.groupby('LAWANAT')['Total'].sum()*100,2)

LAWANAT
AGOLLA         3.25
GUEMERE        3.52
GUIA           2.90
MERINGUE 1     4.58
MERINGUE 2     2.12
MERINGUE 3    12.58
WIDI           3.18
WIDIGUE        2.97
dtype: float64

####  On note des valeurs très différentes dans le lawanat de MERINGUE 3 pour le pourcentage des moins d'un an dans la population. Fluctuation d'échantillonnage ou valeurs aberrantes dans les données du lawanat de MERINGUE 3

### Génération des échantillons sous forme de fichier excel.

In [131]:
echantillon_aleatoire_simple.to_excel('echantillon/echantillon_aleatoire_simple.xlsx')

In [132]:
echantillon_aleatoire_simple_2.to_excel('echantillon/echantillon_aleatoire_simple_2.xlsx')

In [133]:
echantillon_systematique.to_excel('echantillon/echantillon_systematique.xlsx')

In [134]:
echantillon_systematique_2.to_excel('echantillon/echantillon_systematique_2.xlsx')

In [135]:
echantillon_strat_prop.to_excel('echantillon/echantillon_strat_prob.xlsx')

In [136]:
echantillon_strat_eq_nb.to_excel('echantillon/echantillon_strat_eq_nb.xlsx')

In [137]:
echantillon_pondere.to_excel('echantillon/echantillon_pondere.xlsx')

In [138]:
echantillon_en_grappe.to_excel('echantillon/echantillon_en_grappe.xlsx')

# Conclusion

Les enquêtes de couverture au niveau des districts de santé sont possibles à condition de disposer d'une base de sondage. Elles sont une alternative aux données administratives. Cet article nous montre que les différents types d'échantillonnages sont faisables et surtout pertinentes.